In [ ]:
# Elegir generos para el entrenamiento. (Sin Spoken, Old Time, International, Pop)

df_clean = df[df["genre"].isin(['Instrumental', 'Punk', 'Folk', 'Rock', 'Chiptune / Glitch', 'Experimental',
       'Electronic', 'Jazz', 'Classical', 'Ambient Electronic',
       'Blues', 'Easy Listening', 'Soul-RnB', 'Country',  'Hip-Hop'])].drop(columns=['song_id', 'filename', 'filepath'])

In [ ]:
genre_mapping = {
    # 0 - Música Electrónica / Digital
    'Electronic': 0,
    'Chiptune / Glitch': 0,
    'Ambient Electronic': 0,
    'Experimental': 0,
    'Hip-Hop': 0,

    # 1 - Música Instrumental & Tradicional
    'Rock': 1,
    'Punk': 1,
    'Folk': 1,
    'Jazz': 1,
    'Classical': 1,
    'Instrumental': 1,
    'Blues': 1,
    'Easy Listening': 1,
    'Soul-RnB': 1,
    'Country': 1
}


In [ ]:
df_clean['genre_group'] = df_clean['genre'].map(genre_mapping)

In [ ]:
# Definir X e y
X = df_clean.drop(columns=['Chroma_1', 'Chroma_2', 'Chroma_3', 'Chroma_4', 'Chroma_5',
       'Chroma_6', 'Chroma_7', 'Chroma_8', 'Chroma_9', 'Chroma_10',
       'Chroma_11', 'Chroma_12', 'Spectral_contrast_1', 'Spectral_contrast_2',
       'Spectral_contrast_3', 'Spectral_contrast_4', 'Spectral_contrast_5',
       'Spectral_contrast_6', 'Spectral_contrast_7', 'Zero_crossing_rate',
       'Spectral_Rolloff', 'Tempo', 'genre_id', "genre", "genre_group"])
y = df_clean["genre_group"]

# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

# Escalar los datos (CatBoost no lo requiere, pero mantenemos la coherencia)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Definir el modelo CatBoostClassifier
model_xgb = xgb.XGBClassifier(
    n_estimators=200,         # Número de árboles
    learning_rate=0.1,        # Tasa de aprendizaje
    max_depth=10,              # Profundidad de los árboles
    min_child_weight=3,       # Peso mínimo en hojas
    subsample=0.8,            # Uso de datos en cada árbol
    colsample_bytree=0.8,     # Uso de features en cada árbol
    gamma=0.1,                # Regularización de splits
    reg_alpha=0,              # Regularización L1
    reg_lambda=1,             # Regularización L2
    objective='binary:logistic', # Para clasificación binaria
    eval_metric='auc',        # Métrica de evaluación
    random_state=42
)
model_xgb.fit(X_train_scaled, y_train)

# Hacer predicciones
pred = model_xgb.predict(X_test_scaled)

# Evaluar el modelo
print("Accuracy: ", metrics.accuracy_score(y_test, pred))
print(metrics.classification_report(y_test, pred))

# Matriz de confusión
cm = metrics.confusion_matrix(y_test, pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=True, yticklabels=True)

# Personalizar el gráfico
plt.title("Matriz de Confusión - XGBoost")
plt.xlabel("Predicciones")
plt.ylabel("Valores Reales")
plt.show()

